In [ ]:
import h5py
import numpy as np

# Path to your HDF5 file
file_path = '5G_dataset/5G_training_data.hdf5'

# Create an empty dictionary to store NumPy arrays
data_dict = {}

# Open the HDF5 file
with h5py.File(file_path, 'r') as hdf:
    # Iterate over all datasets in the file
    for key in hdf.keys():
        dataset = hdf[key][:]
        data_dict[key] = dataset  # Store each dataset into the dictionary

# Now, you can access individual datasets as NumPy arrays
A_ID = data_dict['A_ID']
B_ID = data_dict['B_ID']
CIR_I = data_dict['CIR_I']
CIR_R = data_dict['CIR_R']
POS_X = data_dict['POS_X']
POS_Y = data_dict['POS_Y']
TD = data_dict['TD']
TD_OFFSET = data_dict['TD_OFFSET']
TIME_STAMP = data_dict['TIME_STAMP']

# For verification, print shapes of arrays
for key, value in data_dict.items():
    print(f"{key} shape: {value.shape}")

In [ ]:
CIR=CIR_R+ 1j*CIR_I

In [ ]:
grt_pos=np.stack((POS_X, POS_Y), axis=1)

In [ ]:
def feature_extraction_F1(cir_matrix):
    R = cir_matrix * np.conj(cir_matrix)
    r = R.flatten()
    r_real = np.real(r)
    r_imag = np.imag(r)
    r_R = np.concatenate((r_real, r_imag))
    feature_vector = r_R / np.linalg.norm(r_R)
    return feature_vector

features_F1_cir = np.array([feature_extraction_F1(cir) for cir in CIR])
print("Shape of Features (F1_cir):", features_F1_cir.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, x1, y_train, y1 = train_test_split(features_F1_cir, grt_pos, test_size=0.4, random_state=42)
X_test, X_pred, y_test, y_pred = train_test_split(x1, y1, test_size=0.5, random_state=42)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
plt.scatter(y_pred[:, 0], y_pred[:, 1], c='blue', label='Ground Truth', alpha=0.5)
plt.title('Ground Truth Positions for the pred set')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test[:, 0], y_test[:, 1], c='blue', label='Ground Truth', alpha=0.5)
plt.title('Ground Truth Positions for the Test Set')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_train[:, 0], y_train[:, 1], c='blue', label='Ground Truth', alpha=0.5)
plt.title('Ground Truth Positions for the train set')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from scipy.special import softmax

def build_grid(grid_size=30, space_bounds=None):
    (xmin, xmax), (ymin, ymax) = space_bounds
    x_lin = np.linspace(xmin, xmax, grid_size)
    y_lin = np.linspace(ymin, ymax, grid_size)
        
    xv, yv = np.meshgrid(x_lin, y_lin)
    grid_points = np.stack([xv.ravel(), yv.ravel()], axis=1)
    return grid_points

def compute_soft_probability_maps(positions, grid_points, temperature=.1):
    N = positions.shape[0]
    K = grid_points.shape[0]
    dists = np.linalg.norm(positions[:, np.newaxis, :] - grid_points[np.newaxis, :, :], axis=2)
    sim_scores = -dists / temperature
    prob_maps = softmax(sim_scores, axis=1)
    return prob_maps

def estimate_positions_from_maps(prob_maps, grid_points):
    return np.matmul(prob_maps, grid_points)

def generate_all_probability_maps(positions, grid_size=30, temperature=.1):
    x_min, x_max = positions[:, 0].min(), positions[:, 0].max()
    y_min, y_max = positions[:, 1].min(), positions[:, 1].max()
    print("Position bounds:", (x_min, x_max), (y_min, y_max))
    grid_points = build_grid(grid_size, space_bounds=((x_min, x_max), (y_min, y_max)))
    print("Grid points shape:", grid_points.shape)
    prob_maps = compute_soft_probability_maps(positions, grid_points, temperature=temperature)

    est_locations = estimate_positions_from_maps(prob_maps, grid_points)

    return prob_maps, est_locations, grid_points

In [ ]:
def plot_heatmap(prob_map, grid_points, title="Heatmap"):
    plt.figure(figsize=(8, 6))
    plt.imshow(prob_map.reshape(100, 100), extent=(grid_points[:, 0].min(), grid_points[:, 0].max(),
                                                  grid_points[:, 1].min(), grid_points[:, 1].max()),
               origin='lower', cmap='hot', alpha=0.7)
    plt.colorbar(label='Probability')
    plt.title(title)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.show()

In [ ]:
prob_map_test, est_locations_test, grid_points_test = generate_all_probability_maps(y_test, grid_size=100, temperature=0.1)
prob_map_train, est_locations_train, grid_points_train = generate_all_probability_maps(y_train, grid_size=100, temperature=0.1)
prob_map_pred, est_locations_pred, grid_points_pred = generate_all_probability_maps(y_pred, grid_size=100, temperature=0.1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test[:, 0], y_test[:, 1], c='blue', label='Ground Truth (Test)', alpha=0.5)
plt.scatter(est_locations_test[:, 0], est_locations_test[:, 1], c='red', label='Estimated (Test)', alpha=0.5)
plt.title('Estimated vs Ground Truth Positions (Test Set)')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from keras import layers
import tensorflow as tf

class NeuralNetwork(tf.keras.Model):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = layers.Dense(512, input_dim=input_size)
        self.bn1 = layers.BatchNormalization()
        self.fc2 = layers.Dense(256)
        self.bn2 = layers.BatchNormalization()
        self.fc4 = layers.Dense(output_size, activation='softmax')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = tf.nn.relu(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)

        x = self.fc4(x)
        return x

# Set output_size to the number of grid points (e.g., 900 for 30x30 grid)
output_size = grid_points_train.shape[0]  # grid_points from your earlier code
print(output_size)

In [ ]:
y_train.shape,X_train.shape

In [ ]:
features_F1_cir.shape[1]

In [ ]:
model=NeuralNetwork(input_size=features_F1_cir.shape[1], output_size=output_size)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_pred, prob_map_pred, epochs=30, batch_size=32, validation_split=0.2)

In [ ]:
model.fit(X_pred, prob_map_pred, epochs=40, batch_size=32, validation_split=0.2)

In [ ]:
model.fit(X_pred, prob_map_pred, epochs=100, batch_size=32, validation_split=0.2)

In [ ]:
predicted_maps = model.predict(X_test)
plot_heatmap(predicted_maps[2], grid_points_test, title="Predicted Heatmap for First Test Position")
print(f"Estimated :- {estimate_positions_from_maps(predicted_maps, grid_points_test)[2]}")
print(f"Ground Truth :- {y_test[2]}")
plot_heatmap(prob_map_test[2], grid_points_test, title="Heatmap for First Test Position")
predicted_positions = np.matmul(predicted_maps, grid_points_test)
mae = np.mean(np.abs(y_test - predicted_positions), axis=0)
print("Mean Absolute Error (Position Predictions):", mae)

In [ ]:
grid_points = build_grid(grid_size=100, space_bounds=((grt_pos[:, 0].min(), grt_pos[:, 0].max()),
                                                    (grt_pos[:, 1].min(), grt_pos[:, 1].max())))
# Plot the grid points
plt.figure(figsize=(10, 10))
plt.scatter(grid_points[:, 0], grid_points[:, 1], c='red', label='Grid Points', alpha=0.5)
plt.scatter(grt_pos[:, 0], grt_pos[:, 1], c='blue', label='Ground Truth', alpha=0.01)
plt.title('Grid Points and Ground Truth Positions')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
predicted_maps.shape, grid_points_test.shape

In [ ]:
jk=estimate_positions_from_maps(predicted_maps, grid_points_test)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test[:, 0], y_test[:, 1], c='blue', label='Ground Truth (Test)', alpha=0.5)
plt.scatter(jk[:, 0], jk[:, 1], c='red', label='Estimated (Test)', alpha=0.5)
plt.title('Estimated vs Ground Truth Positions (Test Set)')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()